In [ ]:
# mini Generative Pre-trained Transformer

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

# ----------------------------
# 샘플 문장 데이터
# ----------------------------
sentences = [
    "안녕하세요 오늘 날씨가 좋네요",
    "저는 학생입니다",
    "오늘은 텐서플로우를 공부합니다",
    "자연어 처리는 재미있습니다",
    "문장을 생성하는 모델을 만들어봅시다"
]

# ----------------------------
# 단어 토크나이저
# ----------------------------
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', lower=False, oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1  # 0번 패딩 포함

sequences = tokenizer.texts_to_sequences(sentences)
max_seq_len = max(len(seq) for seq in sequences)

# 시퀀스 패딩
X_train = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_seq_len, padding='post')
y_train = np.roll(X_train, shift=-1, axis=1)  # 다음 단어 예측
y_train[:, -1] = 0  # 마지막 토큰은 0으로 채우기

# ----------------------------
# Positional Encoding
# ----------------------------
class PositionalEncoding(layers.Layer):
    def __init__(self, max_len, embed_dim):
        super().__init__()
        self.pos_encoding = self.positional_encoding(max_len, embed_dim)

    def positional_encoding(self, max_len, embed_dim):
        positions = np.arange(max_len)[:, np.newaxis]
        dims = np.arange(embed_dim)[np.newaxis, :]
        angle_rates = 1 / np.power(10000, (2 * (dims // 2)) / np.float32(embed_dim))
        angle_rads = positions * angle_rates
        pos_encoding = np.zeros((max_len, embed_dim))
        pos_encoding[:, 0::2] = np.sin(angle_rads[:, 0::2])
        pos_encoding[:, 1::2] = np.cos(angle_rads[:, 1::2])
        return tf.cast(pos_encoding[np.newaxis, ...], tf.float32)

    def call(self, x):
        return x + self.pos_encoding[:, :tf.shape(x)[1], :]

# ----------------------------
# Decoder Block (동적 causal mask)
# ----------------------------
class DecoderBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim):
        super().__init__()
        self.attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation='relu'),
            layers.Dense(embed_dim)
        ])
        self.norm1 = layers.LayerNormalization()
        self.norm2 = layers.LayerNormalization()
        self.dropout1 = layers.Dropout(0.1)
        self.dropout2 = layers.Dropout(0.1)

    def call(self, x, training):
        seq_len = tf.shape(x)[1]
        # 입력 길이에 맞는 causal mask 생성
        mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        mask = mask[tf.newaxis, tf.newaxis, :, :]  # (1,1,seq_len,seq_len)
        attn_output = self.attn(x, x, attention_mask=mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.norm1(x + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.norm2(out1 + ffn_output)

# ----------------------------
# 모델 생성
# ----------------------------
embed_dim = 32
num_heads = 2
ff_dim = 64
num_layers = 2

inputs = layers.Input(shape=(None,), dtype=tf.int32)
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = PositionalEncoding(max_seq_len, embed_dim)(x)

for _ in range(num_layers):
    x = DecoderBlock(embed_dim, num_heads, ff_dim)(x, training=True)

outputs = layers.Dense(vocab_size)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
model.summary()

# ----------------------------
# 학습
# ----------------------------
model.fit(X_train, y_train, epochs=200, batch_size=4, verbose=0)

# ----------------------------
# 문장 생성 함수
# ----------------------------
def generate_sentence(model, start_words, max_len=10):
    seq = tokenizer.texts_to_sequences([start_words])[0]
    for _ in range(max_len):
        padded_seq = tf.expand_dims(seq, 0)  # 배치 차원 추가
        logits = model(padded_seq, training=False)  # mask는 DecoderBlock에서 자동 생성
        next_token = tf.argmax(logits[:, -1, :], axis=-1)[0].numpy()
        if next_token == 0:
            break
        seq.append(next_token)
    return ' '.join(tokenizer.index_word[i] for i in seq)

# ----------------------------
# 문장 생성 예시
# ----------------------------
start_words = "오늘은"
generated_sentence = generate_sentence(model, start_words)
print("Generated:", generated_sentence)


### 제대로 된 GPT를 만들려면 
1. 모델 규모 및 파라미터 수
GPT 모델은, 비록 "미니" 버전일지라도 수백만, 수십억 개의 파라미터를 가집니다. 하지만 이 코드는 레이어 수가 적고 embed_dim (32)이 매우 작아 파라미터 수가 극히 적다. 

2. 학습 데이터
GPT 모델은 책, 기사, 웹사이트 등 인터넷의 방대하고 다양한 텍스트 코퍼스로 학습하게 한다.
현 데이터로는 모델이 의미 있는 언어 지식을 학습하기에 턱없이 부족하며 결과적으로 모델은 제공된 시퀀스를 "암기"하는 수준에 머무르며, 매우 제한적이고 반복적인 출력만을 생성할 수 있습니다.